In [1]:
import sys
sys.path.insert(1, '/Users/punitaverma/Desktop/AI Masters/705.603/workspace/verma-705.603spring24/VIrtualSearch')
import model
from model import Object_Detection_Model
# %reload_ext autoreload
# %load_ext autoreload
# %autoreload 2

In [2]:
weights_path = '/Users/punitaverma/Desktop/AI Masters/705.603/workspace/verma-705.603spring24/VIrtualSearch/lpr-yolov3.weights'
config_path = '/Users/punitaverma/Desktop/AI Masters/705.603/workspace/verma-705.603spring24/VIrtualSearch/lpr-yolov3.cfg' 
yolo_model = Object_Detection_Model(weights_path, config_path)

In [15]:
yolo_model

In [4]:
import os 
import data_pipeline
from data_pipeline import ETL_Pipeline
video_path = '/Users/punitaverma/Desktop/AI Masters/705.603/workspace/verma-705.603spring24/VIrtualSearch/LicensePlateReaderSample_4K.mov'
pipeline = ETL_Pipeline()
data = pipeline.extract(video_path, 30)
# yolo_model.test()

Creating... video_frames/frame0.jpg
Creating... video_frames/frame1800.jpg
Extracted dataframe head:                              0
0     video_frames/frame0.jpg
1  video_frames/frame1800.jpg


In [5]:
data.dtypes

0    object
dtype: object

In [6]:
data_processed = pipeline.transform(data)

video_frames/frame0.jpg
video_frames/frame1800.jpg


In [7]:
y_pred = yolo_model.test(data_processed, [1,2])

Creating report.txt file at  /Users/punitaverma/Desktop/AI Masters/705.603/workspace/verma-705.603spring24/VIrtualSearch/analysis/results


In [8]:
y_pred

[[2404.0, 1570.0, 161.0, 65.0], [1089.0, 1971.0, 229.0, 87.0]]

In [9]:
import os

def get_bounding_box_from_txt(image_name, json_dir):
    txt_file = os.path.join(json_dir, f"{image_name}.txt")
    with open(txt_file, 'r') as f:
        line = f.readline().strip()  # Read the first line
    class_name, *bbox = line.split()  # Split the line by whitespace
    bbox = list(map(float, bbox))  # Convert bbox coordinates to floats
    return bbox

all_bboxes = []
yolo_dir = '/Users/punitaverma/Desktop/AI Masters/705.603/workspace/verma-705.603spring24/VIrtualSearch/outputs'
for file_name in os.listdir(yolo_dir):
    if file_name.endswith("_yolo.txt"):
        image_name = os.path.splitext(file_name)[0]  # Get image name without extension
        bbox = get_bounding_box_from_txt(image_name, yolo_dir)
        all_bboxes.append(bbox)

In [10]:
import pandas as pd

images = '/Users/punitaverma/Desktop/AI Masters/705.603/workspace/verma-705.603spring24/VIrtualSearch/images'
jpeg_images = []
for file_name in os.listdir(images):
    if file_name.lower().endswith(".jpeg"):
        jpeg_images.append(os.path.join(images, file_name))
df = pd.DataFrame({"file_path": jpeg_images})

In [11]:
df

,file_path
0,/Users/punitaverma/Desktop/AI Masters/705.603/...
1,/Users/punitaverma/Desktop/AI Masters/705.603/...
2,/Users/punitaverma/Desktop/AI Masters/705.603/...
3,/Users/punitaverma/Desktop/AI Masters/705.603/...
4,/Users/punitaverma/Desktop/AI Masters/705.603/...
...,...
223,/Users/punitaverma/Desktop/AI Masters/705.603/...
224,/Users/punitaverma/Desktop/AI Masters/705.603/...
225,/Users/punitaverma/Desktop/AI Masters/705.603/...
226,/Users/punitaverma/Desktop/AI Masters/705.603/...


In [12]:
# all_bboxes
# df.iloc[:10]
all_bboxes[0:10]

[[0.6528645833333333,
  0.625462962962963,
  0.10885416666666667,
  0.049074074074074076],
 [0.515625, 0.5958333333333333, 0.1375, 0.06018518518518518],
 [0.40087719298245617, 0.50875, 0.756140350877193, 0.3325],
 [0.463125, 0.8689903846153846, 0.17875, 0.11778846153846154],
 [0.521875, 0.32222222222222224, 0.89375, 0.28888888888888886],
 [0.49375, 0.767578125, 0.27708333333333335, 0.04453125],
 [0.5433333333333333,
  0.5637931034482758,
  0.4777777777777778,
  0.15517241379310345],
 [0.5213004484304933,
  0.8563829787234043,
  0.257847533632287,
  0.09574468085106383],
 [0.6283018867924528,
  0.6105263157894737,
  0.32075471698113206,
  0.14736842105263157],
 [0.511, 0.5228494623655914, 0.278, 0.09408602150537634]]

In [16]:
y_pred = yolo_model.test(df[:10], all_bboxes[0:10])

Creating report.txt file at  /Users/punitaverma/Desktop/AI Masters/705.603/workspace/verma-705.603spring24/VIrtualSearch/analysis/results


In [19]:
import numpy as np

def calculate_iou(bbox_true, bbox_pred):
    """
    Calculate Intersection over Union (IoU) for two bounding boxes.
    """
    # Extract coordinates of the bounding boxes
    x1_true, y1_true, w_true, h_true = bbox_true
    x1_pred, y1_pred, w_pred, h_pred = bbox_pred
    
    # Calculate coordinates of the intersection rectangle
    x1_intersection = max(x1_true, x1_pred)
    y1_intersection = max(y1_true, y1_pred)
    x2_intersection = min(x1_true + w_true, x1_pred + w_pred)
    y2_intersection = min(y1_true + h_true, y1_pred + h_pred)
    
    # Calculate area of intersection rectangle
    intersection_area = max(0, x2_intersection - x1_intersection) * max(0, y2_intersection - y1_intersection)
    
    # Calculate area of union of both bounding boxes
    area_true = w_true * h_true
    area_pred = w_pred * h_pred
    union_area = area_true + area_pred - intersection_area
    
    # Calculate IoU
    iou = intersection_area / union_area
    return iou

def calculate_precision_recall(y_true, y_pred):
    """
    Calculate precision and recall based on true and predicted bounding boxes.
    """
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    
    for bbox_true in y_true:
        # Find the predicted bounding box with the highest IoU
        max_iou = 0
        for bbox_pred in y_pred:
            iou = calculate_iou(bbox_true['bbox'], bbox_pred['bbox'])
            if iou > max_iou:
                max_iou = iou
        
        # If IoU exceeds a certain threshold, consider it a true positive
        if max_iou >= threshold:
            true_positives += 1
        else:
            false_negatives += 1
    
    # Calculate precision and recall
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    
    return precision, recall

def calculate_map(y_true, y_pred):
    """
    Calculate mean Average Precision (mAP) for all classes.
    """
    precisions = []
    recalls = []
    
    # Iterate over each class
    for class_label in range(1):
        # Extract true and predicted bounding boxes for the current class
        y_true_class = [bbox for bbox in y_true if bbox['label'] == class_label]
        y_pred_class = [bbox for bbox in y_pred if bbox['label'] == class_label]
        
        # Calculate precision and recall for the current class
        precision, recall = calculate_precision_recall(y_true_class, y_pred_class)
        precisions.append(precision)
        recalls.append(recall)
    
    # Calculate Average Precision (AP) for each class
    ap_values = []
    for precision, recall in zip(precisions, recalls):
        # Interpolate precision to 11 points
        interpolated_precisions = []
        for recall_threshold in np.linspace(0, 1, 11):
            max_precision = max([p for p, r in zip(precision, recall) if r >= recall_threshold])
            interpolated_precisions.append(max_precision)
        
        # Calculate area under precision-recall curve
        ap = np.mean(interpolated_precisions)
        ap_values.append(ap)
    
    # Calculate mean Average Precision (mAP)
    mAP = np.mean(ap_values)
    
    return mAP

# Example usage:
# y_true = [{'label': label, 'bbox': [x, y, w, h]}, ...] # True bounding boxes
# y_pred = [{'label': label, 'bbox': [x, y, w, h]}, ...] # Predicted bounding boxes
# mAP = calculate_map(y_true, y_pred)


In [20]:
y_pred

[[135.0, 162.0, 216.0, 55.0],
 [83.0, 163.0, 275.0, 290.0],
 [138.0, 187.0, 215.0, 68.0],
 [2.0, 73.0, 244.0, 114.0],
 [285.0, 423.0, 394.0, 125.0],
 [130.0, 245.0, 222.0, 65.0],
 [0, 0, 0, 0],
 [115.0, 263.0, 113.0, 32.0],
 [326.0, 196.0, 261.0, 59.0],
 [291.0, 264.0, 204.0, 41.0]]

In [26]:
calculate_map([int(x) for x in all_bboxes[0:10]], [int(x) for x in y_pred])

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'list'